In [ ]:
import sys
sys.path.append('./')
sys.path.append('./../../../../')
sys.path.append('./../../../../designer')
sys.path.append('./../../../../designer/script')

from spectrum import Spectrum
from film import TwoMaterialFilm
from optimizer.adam import AdamThicknessOptimizer
from optimizer.archive.LM_gradient_descent import LM_optimize_d_simple
from design import BaseDesign

from utils.substitute import optimal_and_thin_film_approx_substitution_onestep_new, equal_optical_thickness

import numpy as np
import matplotlib.pyplot as plt
import copy

: 

In [2]:
wls = np.linspace(400, 700, 300)
wls_more = np.linspace(390, 710, 320)

target_spec_params = 20., wls
target = Spectrum(*target_spec_params, np.ones_like(wls))

d_init = np.loadtxt('./before')
d_eqot_benchmark = np.loadtxt('./after')


film_init = TwoMaterialFilm('Ta2O5_xc', 'SiO2_xc', 'MgF2_xc', d_init)
# xc
film_eqot_benchmark = TwoMaterialFilm('SiO2_xc', 'Ta2O5_xc', 'MgF2_xc', d_eqot_benchmark)
# ours
film_eqot = copy.deepcopy(film_init)
film_optim = copy.deepcopy(film_init)


all_films = [film_init, film_eqot_benchmark, film_eqot, film_optim]
for film in all_films:
    film.add_spec_param(*target_spec_params)

In [3]:
# w/o GD. NOTE: need to wait for compilation of CUDA at first run
equal_optical_thickness(film_eqot, 5.)
# optimal_and_thin_film_approx_substitution_onestep_new(film_optim, 5.)

/root/Thin-Film-Design/working/review/substitution/anti_reflector/./../../../../designer/script/utils/substitute.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
  d[i - 1] += optical_ratio * d[i] + d[i + 1]


1

In [30]:
fig, ax = plt.subplots(1, 1)

ax.plot(wls, film_init.get_spec().get_T(), label='w/ thin layer')
ax.plot(wls, film_eqot_benchmark.get_spec().get_T(), label='eq ot -- xc')
ax.plot(wls, film_eqot.get_spec().get_T(), label='eq ot')
ax.plot(wls, film_optim.get_spec().get_T(), label='optim')
